In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("data3.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96800 entries, 0 to 96799
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   xAcc    96800 non-null  float64
 1   yAcc    96800 non-null  float64
 2   zAcc    96800 non-null  float64
 3   xGyro   96800 non-null  float64
 4   yGyro   96800 non-null  float64
 5   zGyro   96800 non-null  float64
 6   label   96800 non-null  object 
dtypes: float64(6), object(1)
memory usage: 5.2+ MB


### Adjust the features as we want

In [101]:
# every row is an observation of length 400 x 3 x 2

num_rows = data.shape[0]
# Set the step size to select every 400 rows
step = 400

# Create a range of indices to select rows in chunks of 400
indices = np.arange(0, num_rows, step)

# Create a list to store the transposed data
transposed_data = []
new_data = pd.DataFrame()
# Iterate over the original columns
for col in data.columns:
    if col != "label":
        # Extract values from the column and reshape into chunks of 400 rows
        column_values = data[col].values
        # reshaped_values = column_values[indices]
        # Split the list into chunks of 400 values
        chunk_size = 400
        chunks = [column_values[i:i + chunk_size] for i in range(0, len(column_values), chunk_size)]

        # Create a DataFrame
        tmp = pd.DataFrame(chunks, columns=[f'{col}_{i+1}' for i in range(chunk_size)])
        new_data = pd.concat([new_data, tmp], axis=1)

labels = np.array(data.label.iloc[np.arange(0,data.shape[0], 400)])
new_data["label"] = labels

In [103]:
# TEST PER LE SOMME DI VETTORI

# x = np.array(new_data.iloc[0, 1200:1600])
# y = np.array(new_data.iloc[0, 1600:2000])
# z = np.array(new_data.iloc[0, 2000:2400])
# s = x**2 + y**2 + z**2
# s

array([24.8598, 62.301, 45.58670000000001, 8.084599999999998,
       1.7165000000000004, 10.245700000000001, 25.401000000000003,
       57.993399999999994, 36.3779, 13.777400000000002, 35.7806, 85.7354,
       218.48379999999997, 392.6813, 611.6712, 678.1588999999999,
       555.9916000000001, 234.18890000000002, 33.5285, 71.1542, 155.1843,
       132.763, 65.9826, 26.888299999999997, 11.6146, 37.8709, 161.139,
       401.4936, 492.413, 378.2651, 243.3157, 276.9077,
       467.75149999999996, 666.1621, 915.46, 1222.4053000000001,
       1461.3521, 1524.3140999999996, 1421.9522000000002, 1042.4374,
       322.895, 368.60420000000005, 687.6625, 590.4003, 2163.8058,
       4108.514999999999, 2656.7885, 1091.8058, 395.2494000000001,
       772.4921, 1835.0937000000001, 2419.7801000000004, 2484.9901,
       2427.5726000000004, 1747.1200999999999, 1225.6753999999999,
       1157.6475, 2303.0545, 3530.652, 1358.7889, 1314.5949, 2128.7052,
       1546.0115999999998, 733.1405000000001, 236.1301

In [104]:
# every row is an observation of length 400 x 3 x 2

num_rows = new_data.shape[0]
# Set the step size to select every 400 rows
step = 400

# Create a range of indices to select rows in chunks of 400
indices = np.arange(0, num_rows, step)

# Create a list to store the transposed data
new_data2 = pd.DataFrame()
# Iterate over the original columns

def chunk_splitting(row):

    # Split the list into chunks of 400 values
    n_chunks = 2400/400
    chunks = np.array_split(np.array(row.values[:len(row.values)-1], dtype=float), n_chunks)

    return chunks

def acc_sum(vec):
    acc = vec[0]**2 + vec[1]**2 + vec[2]**2
    
    return acc

def gyr_sum(vec):
    gyr = vec[3]**2 + vec[4]**2 + vec[5]**2

    return gyr

chunks = new_data.apply(lambda x: chunk_splitting(x), axis=1)

In [106]:
df = pd.DataFrame()

new_data["acc_sum"] = chunks.apply(lambda x: acc_sum(x))
new_data["gyr_sum"] = chunks.apply(lambda x: gyr_sum(x))
df[[f'acc_{i+1}' for i in range(chunk_size)]] = pd.DataFrame(new_data.acc_sum.to_list(), index = new_data.index)
df[[f'gyr_{i+1}' for i in range(chunk_size)]] = pd.DataFrame(new_data.gyr_sum.to_list(), index = new_data.index)
df

C:\Users\engri\AppData\Local\Temp\ipykernel_4788\4018310215.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[[f'acc_{i+1}' for i in range(chunk_size)]] = pd.DataFrame(new_data.acc_sum.to_list(), index = new_data.index)
C:\Users\engri\AppData\Local\Temp\ipykernel_4788\4018310215.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[[f'acc_{i+1}' for i in range(chunk_size)]] = pd.DataFrame(new_data.acc_sum.to_list(), index = new_data.index)
C:\Users\engri\AppData\Local\Temp\ipykernel_4788\4018310215.py:5: PerformanceWarning

,acc_1,acc_2,acc_3,acc_4,acc_5,acc_6,acc_7,acc_8,acc_9,acc_10,...,gyr_391,gyr_392,gyr_393,gyr_394,gyr_395,gyr_396,gyr_397,gyr_398,gyr_399,gyr_400
0,102.1834,90.9675,84.3782,86.8200,85.8322,92.6537,89.6894,97.3490,98.6358,95.4964,...,1.0601,0.6926,1.2280,2.2769,1.8233,1.9170,1.7334,1.1297,0.9928,1.4838
1,92.9779,88.3341,86.6324,86.8123,91.7411,93.6625,93.5576,92.2101,90.7563,92.9480,...,1.0766,0.5714,1.1667,1.3610,1.9410,1.5113,2.2758,1.6971,1.4322,1.0874
2,93.3540,92.9243,91.7241,91.9994,93.3484,92.1038,90.2842,93.0809,95.1654,93.7950,...,1979.2574,1349.9078,1294.4246,1552.1321,2054.2346,2535.5609,2638.4378,2481.5729,2579.1569,1810.1366
3,89.8760,78.7971,75.3782,78.5737,75.4796,75.7795,92.8978,292.9688,100.0267,64.0433,...,78.6107,32.2398,84.2390,94.2741,57.5363,36.0905,38.2996,30.5021,37.1298,120.4664
4,91.5321,85.0658,93.2414,90.9238,83.2715,77.4394,79.0362,92.8898,238.0556,140.8461,...,667.3516,673.2326,791.8577,989.6654,832.7010,414.1358,98.0451,109.9494,431.3553,2028.9656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,84.0906,86.0024,87.3003,90.2560,82.5494,82.3598,86.0282,128.5545,159.0018,106.0682,...,160.1861,583.5774,197.1283,19.3289,154.6254,1463.6130,2450.3105,2723.1114,3153.2509,2646.7907
238,81.4056,85.9598,84.2938,85.8100,86.5518,98.0753,141.3651,147.4016,100.1646,79.7501,...,140.3972,278.2266,497.7201,715.0805,1307.1282,2740.8501,4541.3252,4320.2798,1517.4874,443.5560
239,62.6009,81.1382,76.0451,102.3529,92.5006,113.1494,102.0984,86.6739,86.1329,92.7734,...,602.1426,773.0960,925.8002,936.6600,922.7966,579.4162,80.8521,33.6214,298.7261,777.3968
240,83.3541,77.5585,82.4261,110.4589,106.6222,96.0413,95.9656,87.8852,91.0178,89.6769,...,162.1006,204.2789,204.0296,160.2385,43.2830,55.6922,230.2677,330.6422,447.0860,864.6298


### Feature extraction

In [20]:
import re
from typing import *
from scipy.fft import fft, fftfreq
from scipy.signal import periodogram
import statsmodels.api as sm
from scipy.stats import skew
from sklearn.model_selection import RandomizedSearchCV,StratifiedKFold
from sklearn.linear_model import LogisticRegression


def maxbin(row: Union[List[float], np.ndarray], plot: bool, n_bins: int = 10) -> List[float]:
    """
    Divide the data into a specified number of bins and calculate the maximum value for each bin.

    Args:
        row (list or array): The data to be divided into bins.
        num_bins (int, optional): The number of bins to divide the data into. Defaults to 10.

    Returns:
        list: A list containing the maximum value for each bin.
    """
    n = len(row)
    freq = list(fftfreq(n*2))
    step = n//n_bins
    max_values = []
    max_freqs = []
    for i in range(0, n+1, step):
        if i==0:
            prec = i
            continue
        mm = max(row[prec: i])
        max_values.append(mm)
        max_freqs.append(freq[list(row).index(mm)])
        prec = i
    if plot:
        return max_freqs, max_values
    else:
        return max_values


def fourier_magnitudes(signal: np.ndarray, n_bins: int = 10, plot: bool = False) -> np.ndarray:
    """
    This function takes a matrix (signal) which contains a time series for each row.
    It applies the Fast Fourier Transform and uses the get_max_per_bin function to find the maximum peaks for each bin of the arrays of the power spectrum for the FFT.

    Args:
        signal (np.ndarray): The input matrix, where each row represents a time series.
        num_bins (int, optional): The number of bins to divide the data into. Defaults to 10.

    Returns:
        np.ndarray: The maximum peaks for each bin of the arrays of the power spectrum for the FFT.
    """
    signal= np.array(signal)
    # let's calculate the FFT
    magnitudes = np.apply_along_axis(fft, 0, signal) 
    # extract just the first half of power spectrum and discover the peaks
    if len(magnitudes.shape)==1:
        magnitudes=magnitudes[:((magnitudes.shape[0])//2)]
    else:
        magnitudes=magnitudes[:((magnitudes.shape[0])//2),:]
    # apply power spectrum formula
    magnitudes = np.abs(magnitudes)**2 
    # take the peaks for each bin
    peaks = np.apply_along_axis(lambda x: maxbin(x, plot, n_bins), 0, magnitudes)
    
    return peaks


def psd_stats(signal: np.ndarray) -> np.ndarray:
    """
    This function takes a matrix (signal) which contains a time series for each row.
    It calculates the Power Spectral Density (PSD) from the signal and returns three different summary statistics
    from the distribution of PSD: the median, the mean absolute deviation, and the skewness (third moment).

    Args:
        signal (np.ndarray): The input matrix, where each row represents a time series.

    Returns:
        np.ndarray: An array containing the median, mean absolute deviation, and skewness of the Power Spectral Density.
    """
    signal= np.array(signal)
    # let's calculate the Power Spectral Density
    psd = np.array(np.abs(np.apply_along_axis(periodogram, 0, signal)[1]), dtype=np.float64)
    
    # extract our main summaries from the distribution
    median = np.median(psd, axis=0)
    # calculate the third moment (skewness)
    third_moment = skew(psd, axis=0)
    # calculate the mean absolute deviation
    mad = np.mean(np.abs(psd - np.mean(psd, axis=0)), axis = 0)
    
    return np.array([median, mad, third_moment])


def acf(signal: np.ndarray, n_lags: int = 10) -> np.ndarray:
    """
    This function takes a matrix (signal) which contains a time series for each row.
    It calculates the auto-correlation for different lags.

    Args:
        signal (np.ndarray): The input matrix, where each row represents a time series.
        num_lags (int, optional): The number of lags to calculate the auto-correlation for. Defaults to 10.

    Returns:
        np.ndarray: The auto-correlation for each time series in the input matrix.
    """
    signal= np.array(signal)
    autocorrelations = np.apply_along_axis(lambda x: sm.tsa.acf(x, nlags=n_lags), 0, signal)

    return autocorrelations


def adjust_df(s: pd.Series) -> pd.DataFrame:
    """
    This function takes a pandas Series, explodes and transposes it, and returns it as a DataFrame.

    Args:
        series (pd.Series): The input pandas Series.

    Returns:
        pd.DataFrame: The adjusted DataFrame.
    """
    df_list = [pd.DataFrame(matrix, index=[index]*len(matrix)) for index, matrix in s.items()]
    new_df = pd.concat(df_list, axis=0)
    new_df.columns = range(new_df.shape[1])
    new_df = new_df.T
    return new_df


def get_season(date):
    if date.month in [3, 4, 5]:
        return 1 # spring
    elif date.month in [6, 7, 8]:
        return 2 # summer
    elif date.month in [9, 10, 11]:
        return 3 # autumn
    else:
        return 4 # winter
    

def preproc(df, n_bins=10, n_lags=10):
    
    # group timeseries by accelerometer (device)
    group_dict = {}
    for col in df.columns:
        match = re.match("^([A-Z1-9]{4})", col)
        group_dict[col] = match.group() if match else None

    new_df = df.T.groupby(group_dict, axis=0)

    # evaluate max magnitudes (intesities of the frequencies from Fourier Fast transform) for each observation for each accelerometer
    magns = new_df.apply(lambda x: fourier_magnitudes(x, n_bins))

    # change col names of our dataframe of magnitudes
    existing_columns = magns.columns
    # generate a list of new column names
    new_columns = []
    i=1
    for j,col_name in enumerate(existing_columns):
        if ((j+1)%(n_bins+1)) == 0:
            i=1
        new_columns.append(f"{col_name}_mag_max_{i}")
        i+=1
    magns.columns = new_columns

    new_df = adjust_df(magns)
    return new_df

In [21]:
prova = preproc(new_data)

C:\Users\engri\AppData\Local\Temp\ipykernel_5748\310717205.py:152: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  magns = new_df.apply(lambda x: fourier_magnitudes(x, n_bins))


In [22]:
prova

""
0
1
2
3
4
...
237
238
239
240
